In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import os

In [3]:
data_types = {'Fiscal Year': int,
              'Employer': str,
              'Initial Approvals': str,
              'Initial Denials': str,
              'Continuing Approvals': str,
              'Continuing Denials': str,
              'NAICS': str,
              'Tax ID': str,
              'State': str,
              'City': str,
              'ZIP': str}

directory = 'data\H1B_aggregate'
h1b_df = pd.DataFrame()
for file in os.listdir(directory):
    temp_df = pd.read_csv(os.path.join(directory, file), header=0, dtype=data_types)
    for col in ['Initial Approvals', 'Initial Denials', 'Continuing Approvals', 'Continuing Denials']:
        temp_df.loc[:, col] = temp_df.loc[:, col].replace(',', '', regex=True)
        temp_df.loc[:, col] = temp_df.loc[:, col].astype(int)
    a = temp_df.loc[:, ['Fiscal Year', 'Tax ID', 'Initial Approvals', 'Initial Denials', 
                              'Continuing Approvals', 'Continuing Denials']].groupby(['Fiscal Year', 'Tax ID']).sum()
    temp_df['TotalApprovals'] = temp_df['Initial Approvals'] + temp_df['Continuing Approvals'] # Get get most active address
    b = temp_df.sort_values('TotalApprovals').groupby(['Fiscal Year', 
                                                       'Tax ID']).first().drop(['Initial Approvals', 
                                                                                'Initial Denials', 
                                                                                'Continuing Approvals', 
                                                                                'Continuing Denials'], axis=1)
    temp_df = a.join(b)
    temp_df['TotalApprovals'] = temp_df['Initial Approvals'] + temp_df['Continuing Approvals']
    h1b_df = h1b_df.append(temp_df)
h1b_df.reset_index().to_csv('data\h1b_aggregated.csv')
h1b_df.head()

Initial Approvals  Initial Denials  Continuing Approvals  \
Fiscal Year Tax ID                                                             
2009        0000                    3                0                     1   
            0001                    4                0                     8   
            0002                    1                0                     2   
            0003                    1                0                     1   
            0004                    4                0                     2   

                    Continuing Denials                           Employer  \
Fiscal Year Tax ID                                                          
2009        0000                     0    KNIGHT INC (A/K/A KINDER MORGAN   
            0001                     2  THE BLAFFER GALLERY UNIVERSITY OF   
            0002                     0            MCDAMS WRIGHT RAGEN INC   
            0003                     0     QUEENS UNIVERSITY OF CHARLOTTE   
            0004                     1                         CALNET INC   

                   NAICS State       City    ZIP  TotalApprovals  
Fiscal Year Tax ID                                                
2009        0000      99    TX    HOUSTON  77002               4  
            0001      71    TX    HOUSTON  77204              12  
            0002      52    WA    SEATTLE  98104               3  
            0003      61    NC  CHARLOTTE  28274               2  
            0004      54    VA     RESTON  20191               6

In [4]:
h1b_df.sort_values('TotalApprovals', ascending=False).head()

,,Initial Approvals,Initial Denials,Continuing Approvals,Continuing Denials,Employer,NAICS,State,City,ZIP,TotalApprovals
Fiscal Year,Tax ID,,,,,,,,,,
2017,4155,3206,410,26176,633,AVANI CONSULTING SVCS INC,54,TX,HOUSTON,77036,29382
2016,4155,3935,196,18570,632,COGNIZANT TECH SOLUTIONS US CORP,54,TX,COLLEGE STATION,77845,22505
2014,4155,5444,709,13209,571,COGNIZANT TECH SOLUTIONS US CORP,54,TX,COLLEGE STATION,77845,18653
2015,4155,3845,320,13174,404,COGNIZANT TECH SOLUTIONS US CORP,54,TX,COLLEGE STATION,77845,17019
2012,4155,9529,605,6628,169,COGNIZANT TECH SOLUTIONS US CORP,54,NJ,TEANECK,07666,16157


In [17]:
naics_codes = pd.read_excel('data/naics_codes.xlsx', usecols=['2017 NAICS US Code', '2017 NAICS US Title'])
naics_codes.columns = ['NAICS', 'NAICS_Title']
naics_codes['NAICS_Title'] = naics_codes['NAICS_Title'].str.strip().str.replace('T$', '', regex=True)
naics_codes.reset_index().to_csv('data/naics_codes.csv')
naics_codes.head()

,NAICS,NAICS_Title
0,11,"Agriculture, Forestry, Fishing and Hunting"
1,111,Crop Production
2,1111,Oilseed and Grain Farming
3,11111,Soybean Farming
4,111110,Soybean Farming


In [21]:
# Zip codes without location information in tableau
no_coord_zips = ['96923', '96401', '95105', '95091', '94814', '94101', '94013', '93885', '93134', '92888', '90821', '85273', '85230', '85222', '75037', 
                 '75036', '67720', '52442', '45043', '39579', '33046', '32729', '10094', '10077', '2216', '00986', '00984', '00981', '00958', '00931', 
                 '00929', '00928', '00922', '00919', '00908', '00737', '00681']

In [47]:
no_coord_zips_df = h1b_df.loc[h1b_df['ZIP'].isin(no_coord_zips)].reset_index().loc[:,['State', 
                                                                                      'City', 
                                                                                      'ZIP']].groupby('ZIP').first()

In [48]:
no_coord_zips_df.to_csv('data\problem_zips.csv')